In [1]:
import os 
#if you want to know current working dir
os.getcwd()
os.chdir('/home/jecosine/Courses/Courses/COMP8800/Projects/CatBoostPlayground')
os.listdir()

from allib.datasets import load_uci, AVAIL_DATASETS
import pandas as pd
import numpy as np
from allib.models import get_pipeline, AVAIL_MODELS
from allib.models.al import get_al_strategy
from allib.metrics import get_metrics
import pickle
import matplotlib.pyplot as plt
from allib.plots import PLMetric
from allib.metrics import distance
from allib.utils import ensure_path

RP = "/home/jecosine/Courses/Courses/COMP8800/Projects/CatBoostPlayground/comp_bs"

In [2]:
make_ppl = get_pipeline('catboost')
ENCODE = 'ordinal'

LOCAL_DIR = "app/comp_bs"
ALSTRATEGY = "typiclust"
SUFFIX = "bs1"
model_extra_params = { "iterations": 100 }

In [2]:
BASE_PATH = "app/comp_bs/"
SUFFIX = "bs100"

def get_post_plot(m, s, pl_metric):
    def post_plot(metric_name, instances, metrics_n_times, strategies, ax, fig):
        for idx, metrics in enumerate(m):
            med, mx, mn = pl_metric.errorbar("std", metrics)
            ax.fill_between(instances, mn, mx, alpha=0.1, color='black')
            ax.plot(instances, med, label=f"baseline(random)", marker="o", linestyle="dashed", markersize=3, c='black')
        ax.legend()
    return post_plot


def plot(
        dataset_name: str,
        model_name: str,
        # strategies: list = None,
        distance_metrics: list=None,
        plot_name: str = None,
        strategy: str = "gsx",
        encode: str = "ordinal"
):
    metrics_n_times = []
    instances = []
    distance_metrics = distance_metrics or list(distance.AVAIL_DIST_METRICS.keys())
    plot_name = plot_name if plot_name is not None else f"{dataset_name}@{model_name}@{strategy}@{encode}@{SUFFIX}.png"
    # if ensure_path(os.path.join("./plots/pl_metric", plot_name), False):
    #     print(f"plot {plot_name} already exists, continue")
    #     return
    # for strategy in strategies:
    for metric in distance_metrics:
        stats = None
        cache_name = f"{dataset_name.replace('/', '_')}@{model_name}@{strategy}_{metric}_{encode}@{SUFFIX}@x20.pkl"
        if not os.path.isfile(os.path.join(BASE_PATH + "ppl_cache", cache_name)):
            print(f"exp {cache_name} does not exist, continue")
            continue
        with open(os.path.join(BASE_PATH + "ppl_cache", cache_name), "rb") as f:
            stats = pickle.load(f)
        metrics_n_times.append([stats[i]["accuracy"] for i in range(len(stats))])
        instances = np.array(stats[0]["instances"], dtype=int)
    baseline_euclidean = None
    baseline_cosine = None
    for metric in ["cosine"]:
        stats = None
        cache_name = f"{dataset_name.replace('/', '_')}@{model_name}@random_{metric}_{encode}@{SUFFIX}@x20.pkl"
        if not os.path.isfile(os.path.join(BASE_PATH + "ppl_cache", cache_name)):
            print(f"exp {cache_name} does not exist, continue")
            continue
        with open(os.path.join(BASE_PATH + "ppl_cache", cache_name), "rb") as f:
            stats = pickle.load(f)
        metrics_n_times.append([stats[i]["accuracy"] for i in range(len(stats))])

    pl_metric = PLMetric(output_path="app/comp_bs/plots/")
    pl_metric.plot(
        metric_name="F1",
        instances=instances,
        metrics_n_times=np.array(metrics_n_times[:-1]),
        strategies=distance_metrics,
        plot_name=plot_name,
        cmap=plt.get_cmap("Paired"),
        dpi=300,
        title=f"F1 {dataset_name}@{model_name}@{strategy}",
        errorbar="std",
        post_plot=get_post_plot(np.array(metrics_n_times[-1:]), ["cosine"], pl_metric)
    )

In [3]:
for dsn in ["adult"]:
    plot(dsn, "catboost", distance_metrics=list(distance.AVAIL_DIST_METRICS.keys()), strategy="typiclust", encode="ordinal")


exp adult@catboost@typiclust_lin1_ordinal@bs100@x20.pkl does not exist, continue
exp adult@catboost@typiclust_goodall1_ordinal@bs100@x20.pkl does not exist, continue
exp adult@catboost@typiclust_goodall2_ordinal@bs100@x20.pkl does not exist, continue
exp adult@catboost@typiclust_goodall3_ordinal@bs100@x20.pkl does not exist, continue
exp adult@catboost@typiclust_goodall4_ordinal@bs100@x20.pkl does not exist, continue
exp adult@catboost@typiclust_smirnov_ordinal@bs100@x20.pkl does not exist, continue
exp adult@catboost@typiclust_gambaryan_ordinal@bs100@x20.pkl does not exist, continue
exp adult@catboost@typiclust_burnaby_ordinal@bs100@x20.pkl does not exist, continue
exp adult@catboost@typiclust_anderberg_ordinal@bs100@x20.pkl does not exist, continue
